import all libraries
create class 

In [87]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.utils import resample

python class for normal logistic regression

In [88]:
class LogisticRegression:
    def __init__(self, weights=None, bias=None, learningRate=0.01, epochs=1000,selectedFeatures=None):
        self.weights = weights
        self.bias = bias
        self.learningRate = learningRate
        self.epochs = epochs
        self.selectedFeatures = selectedFeatures
    

    def loss (self, y, y_hat):
        return -np.mean(y * np.log(y_hat) + (1 - y) * np.log(1 - y_hat))
    
    def __sigmoid(self, z):
        return 1 / (1 + np.exp(-z))

    def fit(self, X, y):

        numOfSamples = X.shape[0]
        numOfFeatures = X.shape[1]
        weights = np.zeros(numOfFeatures)
        bias = 0
        learningRate = 0.01
        epochs = 1000
        selectedFeatures = np.random.choice(numOfFeatures, 2, replace=False)
        print(selectedFeatures)

        for i in range(epochs):
            y_hat = self.__sigmoid(np.dot(X, weights) + bias)
            dw = (1 / numOfSamples) * np.dot(X.T, (y_hat - y))
            db = (1 / numOfSamples) * np.sum(y_hat - y)
            weights -= learningRate * dw
            bias -= learningRate * db

            if i % 100 == 0:
                print(f'Epoch {i} loss: {self.loss(y, y_hat)}')

        self.weights = weights
        self.bias = bias
        self.selectedFeatures = selectedFeatures

    def predict(self, X):
        y_hat = self.__sigmoid(np.dot(X, self.weights) + self.bias)
        return y_hat
    
    def calculate_matrics(self, y, y_hat):
        y_pred = np.round(y_hat)
        accuracy = np.mean(y == y_pred)
        return accuracy
    
    

In [89]:
def dataPreProcessing_creditcard():
    df = pd.read_csv('../Dataset/creditcard.csv')
    numOfAttribuetes = df.shape[1]
    numOfRecords = df.shape[0]
    print(f'Number of records: {numOfRecords}')
    print(f'Number of attributes: {numOfAttribuetes}')
    # print(df.columns)
    # print(df.head(10))
    # print(df.describe())
    # print(df.info())

    #1 Checking for missing values  -----> use one less sum to see detailed missing values
    missingValues = df.isnull().sum().sum()
    # print(f'Missing values: {missingValues}')
    #2 Checking for duplicates
    duplicates = df.duplicated().sum()
    # print(f'Duplicates: {duplicates}')
    # duplicateRows = df[df.duplicated()]
    # print(f'Duplicate rows: {duplicateRows}')

    #3 replace missing values with mean ( numerical attributes)
    df.fillna(df.mean(numeric_only=True), inplace=True)

    #4 remove exact duplicates rows, keep the first one
    df.drop_duplicates(inplace=True)
    print(f'Number of records after removing duplicates: {df.shape[0]}')

    #5 if any target value is missing, remove the record
    df.dropna(subset=['Class'], inplace=True)
    print(f'Number of records after removing missing target values: {df.shape[0]}')

    #6 split the data into features and target
    target_column = 'Class' 
    X = df.drop(target_column, axis=1)
    y = df[target_column]

    # print(X.shape)
    # print(y.shape)
    # print(y.value_counts())
    # print(X.columns)

    #7 display non numerical attributes
    nonNumericalAttributes = X.select_dtypes(include=['object']).columns
    # print(f'Non numerical attributes: {nonNumericalAttributes}')

    # 8 scale the features
    from sklearn.preprocessing import StandardScaler
    scaler = StandardScaler()
    scaled_X = scaler.fit_transform(X)

    #9 make x a dataframe again
    X = pd.DataFrame(scaled_X, columns=X.columns)

    return X, y

In [90]:
def test_train_split(X, y, test_size=0.2):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    return X_train, X_test, y_train, y_test

main Function, make changes here to slect different dataset

DataSetLocation:
../Dataset/creditcard.csv

In [91]:
def main():

# 1 - Data Preprocessing for creditcard dataset
    X,y = dataPreProcessing_creditcard()
    # print (X)
    # print(X.shape)
    # print(y.shape)
    # print(y.value_counts())
    # print(X.columns)

    X_train, X_test, y_train, y_test = test_train_split(X, y, test_size=0.2)
    
# create 9 datasets smapling with replacement

    data_sets = []
    for i in range(9):
        X_train_sampled, y_train_sampled = resample(X_train, y_train, replace=True)
        data_sets.append((X_train_sampled, y_train_sampled))
    # print(data_sets)

    bagging_models=[]
    for i in range(9):
        model = LogisticRegression()
        model.fit(data_sets[i][0], data_sets[i][1])
        bagging_models.append(model)
    # print(bagging_models)

    # create a meta classifier
    meta_classifier = LogisticRegression()



    
if __name__ == '__main__':
    main()

Number of records: 284807
Number of attributes: 31
Number of records after removing duplicates: 283726
Number of records after removing missing target values: 283726
[21  3]
Epoch 0 loss: 0.6931471805599451
Epoch 100 loss: 0.49646814361530983
Epoch 200 loss: 0.3743536761500963
Epoch 300 loss: 0.29502269842331735
Epoch 400 loss: 0.24088481823669305
Epoch 500 loss: 0.20225386382772237
Epoch 600 loss: 0.17361912423277487
Epoch 700 loss: 0.15170716520512859
Epoch 800 loss: 0.13448760273378152
Epoch 900 loss: 0.12064997352287965
